<a href="https://colab.research.google.com/github/AbhiT1196/Training_Material/blob/main/Copy_of_HealthCare_Chatbot_ForCommon_Medications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install transformers faiss-cpu sentence-transformers


  Using cached faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-

In [2]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00


In [3]:
pip install notebook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.2 MB/s eta 0:00:00


In [7]:
import os
import faiss
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Step 1: Loading the pre-trained models
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')
generator_model = pipeline('text-generation', model='gpt2')

# Step 2: Function to extract text from a single PDF file
def extract_text_from_pdf(pdf_file):
    documents = []
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text = page.extract_text()
            if text:  # Ensure that text is not None
                documents.append(text)
    return documents

# Step 3: Loading documents from a specific PDF file
pdf_file = r'/content/Common Medications and Health Information File.pdf'  # Path of File
documents = extract_text_from_pdf(pdf_file)

# Debugging: Printing Text from extracted documents
if not documents:
    print("No text was extracted from the PDF.")
else:
    print(f"Extracted {len(documents)} pages of text.")

# Step 4: Creating embeddings for the documents
if documents:  # Only create embeddings if documents are not empty
    document_embeddings = retriever_model.encode(documents)

    # Step 5: Creating a FAISS index
    dimension = document_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(document_embeddings).astype('float32'))

    # Step 6: Defining a function to retrieve relevant documents
    def retrieve_documents(query, k=2):
        query_embedding = retriever_model.encode([query])
        distances, indices = index.search(np.array(query_embedding).astype('float32'), k)
        return [documents[i] for i in indices[0]]

    # Step 7: Defining a function to generate a response
    def generate_response(query):
        retrieved_docs = retrieve_documents(query)
        context = " ".join(retrieved_docs)
        response = generator_model(f"Context: {context}\nQuery: {query}\nResponse:", max_new_tokens=50)
        return response[0]['generated_text']

    # Step 8: Chatbot
    if __name__ == "__main__":
        user_query = "What are the symptoms of the flu?"
        response = generate_response(user_query)
        print("Chatbot Response:", response)
else:
    print("Exiting: No Information Available.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Extracted 1 pages of text.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot Response: Context: 1. Aspirin
Uses: Aspirin is commonly used to reduce fever, relieve mild to moderate
pain, and reduce inflammation. It is also used in low doses to reduce the
risk of heart attacks and strokes. Dosage: For pain relief, the typical
dose for adults is 325-650 mg every 4 to 6 hours as needed. Do not exceed
4,000 mg in a 24-hour period. Side Effects: Possible side effects include
stomach upset, heartburn, and increased risk of bleeding.
2. Ibuprofen
Uses: Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID) used for
pain relief, reducing inflammation, and lowering fever.
Dosage: The usual dose for adults is 200-400 mg every 4 to 6 hours as
needed. Do not exceed 1,200 mg in a 24-hour period without a doctor's
advice. Side Effects: Common side effects include stomach pain, nausea,
and dizziness.
3. Symptoms of Common Cold
Overview: The common cold is caused by viral infections and is
characterized by symptoms such as:
Runny or stuffy nose
Sore throat
Cough
Sne